# This code is the basic code that was used for the heat source radii determination in the paper "Establishing an automated heat-source calibration framework, Rissaki et al, 2023"

# With some modifications, it could be used for automatic heat source radii determination of any welding case

# Please read through the code and if you have any questions contact Dimitra Rissaki by email dimrissaki@gmail.com


In [ ]:
#import required libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import interp1d
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
%matplotlib inline
%matplotlib notebook
from mpl_toolkits.mplot3d import axes3d
import math

# display all rows of dataframes
pd.set_option('display.max_rows', None)

# Read experimental fusion boundary file
The code below was written to read the experimental fusion boundary from an output file of FEAT-WMT program. In case of another experimental fusion boundary file the below code should be modified accordingly.

In [ ]:
# define the path of fusion boundary files (modify the path according to your path)
path = '/Users/dimitra/Dropbox (The University of Manchester)/Dimitra_Rissaki/07_Research/01_Heat_Source_Calibration_Exploration/heat_source_exploration/_1st_pass/efficiency_fixed_74/FEAT_output_files/fb_files_txt'

# find experimental fusion boundary from one (any) fusion boundary file
file = (path + "/" + 'S-FUS_10_10_10.txt')

lines = []
with open(file,'r') as f:
    lines = f.readlines()

# extract from text file the experimental fusion boundary
fb_exper = lines[59:466]

# remove 'L   8   2\n' lines
fb_exper = list(filter(('L   8   2\n').__ne__, fb_exper))

# convert the list into dataframe
fb_exper = [x.strip() for x in fb_exper]
fb_exper = pd.DataFrame(fb_exper)
fb_exper = fb_exper[0].str.split(" ", 2, expand=True)

# delete the last column of zeros
fb_exper = fb_exper.drop(fb_exper.columns[2], axis=1)

# convert string to float
fb_exper[0] = fb_exper[0].astype(float)
fb_exper[1] = fb_exper[1].astype(float)

# drop dublicate rows
fb_exper =  fb_exper.drop_duplicates()

# plot the experimental fusion boundary
plt.plot(fb_exper.iloc[:,0],fb_exper.iloc[:,1],'.')

In [ ]:
# define right and left experimental fusion boundary profile
fb_exper_right = fb_exper.loc[fb_exper[0]>0]
fb_exper_left = fb_exper.loc[fb_exper[0]<=0]

# Simulation and experimental fusion boundary comparison
The code below was written to compare the experimental fusion boundary with a simulation fusion boundary generated by FEAT-WMT program. In case of another program used for simulation, the code below should be changed accordingly.

In [ ]:
def fb_comparison(ra, rl, rv, fb_exper_right, fb_exper_left):
    
    # Create empty list to store metric value of loop for each case
    results = []
    
    # for loop to read each of the text files
    for i in ra:
        for j in rl:
            for k in rv:
                r_a = str(i)
                r_l = str(j)
                r_v = str(k)
                filename = "S-FUS_" + r_a + "_" + r_l + "_" + r_v + ".txt"
                file = (path + "/" + filename)

                lines = []
                with open(file,'r') as f:
                    lines = f.readlines()

                # find line in text file where simulation fusion boundary starts
                lookup = 'START_OBJECT  -1 CONTOUR_ELEMENTS_TFUSION'
                with open(file) as my_file:
                    for num, line in enumerate(my_file, 1):
                        if lookup in line:
                            index_start = num

                # find line in text file where simulation fusion boundary stops
                lookup = 'START_OBJECT  -1 CONTOUR_ELEMENTS_TLIQ'
                with open(file) as my_file:
                    for num, line in enumerate(my_file, 1):
                        if lookup in line:
                            index_stop = num - 2

                # simulation fusion boundary
                fb_simul = lines[index_start:index_stop]

                # remove lines which contain other than fusion boundary information
                fb_simul = list(filter(('L   5   2\n').__ne__, fb_simul)) # remove 'L   5   2\n' lines
                fb_simul = list(filter(('L   5   3\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   4\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   5\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   6\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   7\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   8\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5   9\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5  10\n').__ne__, fb_simul)) 
                fb_simul = list(filter(('L   5  11\n').__ne__, fb_simul)) 

                # convert the list into dataframe
                fb_simul = [x.strip() for x in fb_simul]
                fb_simul = pd.DataFrame(fb_simul)
                fb_simul = fb_simul[0].str.split(" ", 2, expand=True)
                
                # delete the last column of zeros
                fb_simul = fb_simul.drop(fb_simul.columns[2], axis=1)

                # convert string to float
                fb_simul[0] = fb_simul[0].astype(float)
                fb_simul[1] = fb_simul[1].astype(float)

                # sort columns of right axisymmetric profile by ascending order of y'y
                fb_simul = fb_simul.sort_values([1])

                # drop dublicate rows
                fb_simul = fb_simul.drop_duplicates()
                
                # reset index of dataframe
                fb_simul = fb_simul.reset_index(drop = True)

                # find symmetric profile in y'y axis
                fb_simul_sym = fb_simul.copy()
                fb_simul_sym[0] = fb_simul_sym[0].apply(lambda x: x*-1)

                # convert string to float
                fb_simul_sym[0] = fb_simul_sym[0].astype(float)
                fb_simul_sym[1] = fb_simul_sym[1].astype(float)

                # sort columns of left axisymmetric profile by descending order of y'y
                fb_simul_sym = fb_simul_sym.sort_values([1],ascending = False)

                # drop dublicate rows
                fb_simul_sym = fb_simul_sym.drop_duplicates()
                
                # reset index of dataframe
                fb_simul_sym = fb_simul_sym.reset_index(drop = True)

                # merge the two half axisymmetrical fusion boundary shapes
                fb_simul_whole = pd.concat([fb_simul_sym, fb_simul])

                #plot of simulation fusion boundary shape
                fig_sim = plt.plot(fb_simul_whole.iloc[:,0],fb_simul_whole.iloc[:,1])

                ##########################################################################################
                # MSE in horizontal axis between right half of simulation and experimental fusion boundary
                ##########################################################################################
                # Find interpolation function of right half of simulation fusion boundary
                # The x,y axis should be flipped so for the half profile to form a function  
                x_sim = fb_simul.iloc[:,1] # x is the vertical axis
                for n in range(len(x_sim)-1): # the x should not have dublicates for the interpolation to work
                    if x_sim[n] == x_sim[n+1]: # thus modify one of the dublicates by a small value
                        x_sim[n+1] = x_sim[n+1] + 0.0001          
                y_sim = fb_simul.iloc[:,0] # y is the horizontal axis
                f_sim = interp1d(x_sim, y_sim) # define a function that will find any point on the interpolated curve
                interp1d(x_sim, y_sim, kind = 'cubic') # apply cubic interpolation

                # Find interpolation function of right half of experimental fusion boundary
                # The x,y axis should be flipped so for the half profile to form a function
                fb_exper_right = fb_exper_right.reset_index(drop = True)
                x_exp = fb_exper_right.iloc[:,1] # x is the vertical axis
                y_exp = fb_exper_right.iloc[:,0] # y is the horizontal axis
                f_exp = interp1d(x_exp, y_exp) # define a function that will find any point on the interpolated curve
                interp1d(x_exp, y_exp, kind = 'cubic') # apply cubic interpolation

                # Find interval of interpolation (100 points in vertical axis)
                x1 = max(min(x_sim),min(x_exp)) # minimum value of interpolation interval
                x2 = min(max(x_sim),max(x_exp)) # maximum value of interpolation interval
                points = np.linspace(x1, x2, num = 100) # perform interpolation

                # Calculate mean square difference in horizontal axis
                mse_horiz_right = sum((f_sim(points) - f_exp(points))**2)/100

                ##########################################################################################
                # MSE in horizontal axis between left half of simulation and experimental fusion boundary
                ##########################################################################################
                # Find interpolation function of left half of simulation fusion boundary
                # The x,y axis should be flipped so for the half profile to form a function  
                x_sim = fb_simul_sym.iloc[:,1] # x is the vertical axis
                for n in range(len(x_sim)-1): # the x should not have dublicates for the interpolation to work
                    if x_sim[n] == x_sim[n+1]: # thus modify one of the dublicates by a small value
                        x_sim[n+1] = x_sim[n+1] + 0.0001  
                y_sim = fb_simul_sym.iloc[:,0] # y is the horizontal axis
                f_sim = interp1d(x_sim, y_sim) # define a function that will find any point on the interpolated curve
                interp1d(x_sim, y_sim, kind = 'cubic') # apply cubic interpolation

                # Find interpolation function of left half of experimental fusion boundary
                # The x,y axis should be flipped so for the half profile to form a function
                fb_exper_left = fb_exper_left.reset_index(drop = True)
                fb_exper_left.iloc[17,1] = fb_exper_left.iloc[17,1] - 0.001 # I change a little bit the one line because in the 2nd column all values should be different otherwise interpolation is not running
                x_exp = fb_exper_left.iloc[:,1] # x is the vertical axis
                y_exp = fb_exper_left.iloc[:,0] # y is the horizontal axis
                f_exp = interp1d(x_exp, y_exp) # define a function that will find any point on the interpolated curve
                interp1d(x_exp, y_exp, kind = 'cubic') # apply cubic interpolation

                # Find interval of interpolation (100 points in vertical axis)
                x1 = max(min(x_sim),min(x_exp)) # minimum value of interpolation interval
                x2 = min(max(x_sim),max(x_exp)) # maximum value of interpolation interval
                points = np.linspace(x1, x2, num = 100) # perform interpolation

                # Calculate mean square difference in horizontal axis
                mse_horiz_left = sum((f_sim(points) - f_exp(points))**2)/100

                ###############################################################################################
                # MSE in vertical axis between right half bottom of simulation and experimental fusion boundary
                ###############################################################################################
                # Find interpolation function of right half of simulation fusion boundary 
                fb_simul = fb_simul.reset_index(drop = True) # reset index of dataframe
                index_max = fb_simul[0].idxmax()
                fb_simul_bottom = fb_simul.iloc[:index_max,:] # keep only bottom profile, so to be a function (one y value for each x)
                fb_simul_bottom = fb_simul_bottom.reset_index(drop = True) # reset index of dataframe
                x_sim = fb_simul_bottom.iloc[:,0] # x is the horizontal axis
                for n in range(len(x_sim)-1): # the x should not have dublicates for the interpolation to work
                    if x_sim[n] == x_sim[n+1]: # thus modify one of the dublicates by a small value
                        x_sim[n+1] = x_sim[n+1] + 0.0001 
                y_sim = fb_simul_bottom.iloc[:,1] # y is the vertical axis
                f_sim = interp1d(x_sim, y_sim) # define a function that will find any point on the interpolated curve
                interp1d(x_sim, y_sim, kind = 'cubic') # apply cubic interpolation

                # Find interpolation function of right half of experimental fusion boundary
                fb_exper_right = fb_exper_right.reset_index(drop = True) # reset index of dataframe
                index_max = fb_exper_right[0].idxmax()
                fb_exper_right_bottom = fb_exper_right.iloc[:index_max,:] # keep only bottom profile, so to be a function (one y value for each x)
                x_exp = fb_exper_right_bottom.iloc[:,0] # x is the horizontal axis
                y_exp = fb_exper_right_bottom.iloc[:,1] # y is the vertical axis
                f_exp = interp1d(x_exp, y_exp) # define a function that will find any point on the interpolated curve
                interp1d(x_exp, y_exp, kind = 'cubic') # apply cubic interpolation

                # Find interval of interpolation (100 points in vertical axis)
                x1 = max(min(x_sim),min(x_exp)) # minimum value of interpolation interval
                x2 = min(max(x_sim),max(x_exp)) # maximum value of interpolation interval
                points = np.linspace(x1, x2, num = 100) # perform interpolation

                # Calculate mean square difference in vertical axis
                mse_vertic_right = sum((f_sim(points) - f_exp(points))**2)/100

                ###############################################################################################
                # MSE in vertical axis between left half bottom of simulation and experimental fusion boundary
                ###############################################################################################
                # Find interpolation function of left half of simulation fusion boundary 
                fb_simul_sym = fb_simul_sym.sort_values([1]) # sort columns of left axisymmetric profile by ascending order of y'y
                fb_simul_sym = fb_simul_sym.reset_index(drop = True) # reset index of dataframe
                index_min = fb_simul_sym[0].idxmin()
                fb_simul_sym_bottom = fb_simul_sym.iloc[:index_min,:] # keep only bottom profile, so to be a function (one y value for each x)
                fb_simul_sym_bottom = fb_simul_sym_bottom.reset_index(drop = True) # reset index of dataframe
                x_sim = fb_simul_sym_bottom.iloc[:,0] # x is the horizontal axis
                for n in range(len(x_sim)-1): # the x should not have dublicates for the interpolation to work
                    if x_sim[n] == x_sim[n+1]: # thus modify one of the dublicates by a small value
                        x_sim[n+1] = x_sim[n+1] + 0.0001 
                y_sim = fb_simul_sym_bottom.iloc[:,1] # y is the vertical axis
                f_sim = interp1d(x_sim, y_sim) # define a function that will find any point on the interpolated curve
                interp1d(x_sim, y_sim, kind = 'cubic') # apply cubic interpolation

                # Find interpolation function of left half of experimental fusion boundary
                fb_exper_left = fb_exper_left.sort_values([1]) # sort columns of left axisymmetric profile by ascending order of y'y
                fb_exper_left = fb_exper_left.reset_index(drop = True) # reset index of dataframe
                index_min = fb_exper_left[0].idxmin()
                fb_exper_left_bottom = fb_exper_left.iloc[:index_min,:] # keep only bottom profile, so to be a function (one y value for each x)
                x_exp = fb_exper_left_bottom.iloc[:,0] # x is the horizontal axis
                y_exp = fb_exper_left_bottom.iloc[:,1] # y is the vertical axis
                f_exp = interp1d(x_exp, y_exp) # define a function that will find any point on the interpolated curve
                interp1d(x_exp, y_exp, kind = 'cubic') # apply cubic interpolation

                # Find interval of interpolation (100 points in vertical axis)
                x1 = max(min(x_sim),min(x_exp)) # minimum value of interpolation interval
                x2 = min(max(x_sim),max(x_exp)) # maximum value of interpolation interval
                points = np.linspace(x1, x2, num = 100) # perform interpolation

                # Calculate mean square difference in vertical axis
                mse_vertic_left = sum((f_sim(points) - f_exp(points))**2)/100

                ##############################################################################
                # Combine MSE of left-right and horizontal-vertical fusion boundary comparison
                ##############################################################################
                mse_vertic = (mse_vertic_right + mse_vertic_left)/2
                mse_horiz = (mse_horiz_right + mse_horiz_left)/2
                mse_fb = (mse_vertic + mse_horiz)/2
                
                result = [mse_vertic_right, mse_vertic_left, mse_vertic, mse_horiz_right, mse_horiz_left, mse_horiz, mse_fb, r_a, r_l, r_v]     
                results.append(result)
                
    # plot of experimental fusion boundary
    fig_exp = plt.plot(fb_exper.iloc[:,0],fb_exper.iloc[:,1],'.')          
    
    results_df = pd.DataFrame(results, columns = ['mse_vertic_right', 'mse_vertic_left', 'mse_vertic', 'mse_horiz_right', 'mse_horiz_left', 'mse_horiz', 'mse_fb', 'r_a', 'r_l', 'r_v'])

    return results_df, fig_sim


In [ ]:
# select the values for the heat source radii for which to calculate the fusion boundary comparison metric
# (modify the heat source radii values according to your values)
ra = [10,20,30]
rl = [10,20,30]
rv = [10,20,30]

# calculate the fusion boundary comparison metric
results_fb = fb_comparison(ra, rl, rv, fb_exper_right, fb_exper_left)
                            

# 3D plots to view the value of fusion boundary comparison metric for all different radii

In [ ]:
# axes are ra, rl, rv and the colour bar shows the value of fusion boundary metric
fig = plt.figure()
ax = plt.axes(projection='3d')

x = results_fb['r_a']
x = pd.to_numeric(x)
x = x/10
y = results_fb['r_l']
y = pd.to_numeric(y)
y = y/10
z = results_fb['r_v']
z = pd.to_numeric(z)
z = z/10
c = results_fb['mse_fb']

img = ax.scatter(x, y, z, c=c, cmap='YlOrRd', alpha=1)
ax.set_xlabel('$ra$')
ax.set_ylabel('$rl$')
ax.set_zlabel('$rv$')
fig.colorbar(img)
plt.show()


In [ ]:
# axes are rl, rv, value of fusion boundary metric and the colour bar shows the value of fusion boundary metric as well
fig = plt.figure()
ax = plt.axes(projection='3d')

x = results_fb['r_l']
x = pd.to_numeric(x)
x = x/10
y = results_fb['r_v']
y = pd.to_numeric(y)
y = y/10
z = results_fb['mse_fb']
z = pd.to_numeric(z)
c = results_fb['mse_fb']

img = ax.scatter(x, y, z, c=c, cmap='YlOrRd', alpha=1)
ax.set_xlabel('$rl$')
ax.set_ylabel('$rv$')
ax.set_zlabel('$MSE$')
fig.colorbar(img)
plt.show()


In [ ]:
# find solution with smallest value of fusion boundary metric
print(results_fb[results_fb.mse_fb == results_fb.mse_fb.min()])

In [ ]:
# print 10 best solutions (solutions with smallest value of fusion boundary metric)
n_best = results_fb.nsmallest(10, 'mse_fb')
n_best

In [ ]:
# plot simulation fusion boundary of best solution (best ra, rl, rv)
# (modify the best solution according to the best solution you found)
fb_comparison([29], [22], [15], fb_exper_right, fb_exper_left)